In [1]:
!pip install nltk faiss-cpu sentence-transformers transformers pillow torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 65.5 MB/s eta 0:00:00


In [2]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    corpus = []

    for sentence in sentences:
        text = re.sub('[^a-zA-Z]', ' ', sentence)
        text = text.lower()
        words = text.split()
        words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
        corpus.append(" ".join(words))

    return corpus


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
knowledge_text = """
Elon Musk is the CEO of Tesla and SpaceX.
SpaceX focuses on reusable rockets and space missions.
Tesla develops electric vehicles and clean energy solutions.
"""

corpus = preprocess_text(knowledge_text)
print(corpus)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(corpus)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Vector DB size:", index.ntotal)


In [ ]:
def retrieve_answer(query):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), 1)
    return corpus[indices[0][0]]


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
image_model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)

def caption_image(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = processor(image, return_tensors="pt")
    output = image_model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
image_path = "/content/sample_image.jpg"  # upload your image
caption = caption_image(image_path)

print("Image Caption:", caption)


NameError: name 'caption_image' is not defined

In [ ]:
def multimodal_chatbot(user_text=None, image_path=None):
    combined_input = ""

    if user_text:
        combined_input += user_text + " "

    if image_path:
        image_caption = caption_image(image_path)
        combined_input += image_caption

    response = retrieve_answer(combined_input)
    return response


In [ ]:
response = multimodal_chatbot(
    user_text="What is this related to?",
    image_path="/content/sample_image.jpg"
)

print("Chatbot Response:", response)
